In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
import keras_tuner as kt

#  Import and read the charity_data.csv.
import pandas as pd 
df = pd.read_csv('Resources/cleaned_data.csv', delimiter=',')
df.head()

,id,age_days,age_years,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,0,18393,50,2,168,62.0,110,80,1,1,0,0,1,0
1,1,20228,55,1,156,85.0,140,90,3,1,0,0,1,1
2,2,18857,52,1,165,64.0,130,70,3,1,0,0,0,1
3,3,17623,48,2,169,82.0,150,100,1,1,0,0,1,1
4,4,17474,48,1,156,56.0,100,60,1,1,0,0,0,0


# Dataset Exploration

In [2]:
df.shape

(70000, 14)

In [3]:
df.describe()

,id,age_days,age_years,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
count,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000
mean,49972.419900,19468.865814,53.303157,1.349571,164.359229,74.205690,128.817286,96.630414,1.366871,1.226457,0.088129,0.053771,0.803729,0.499700
std,28851.302323,2467.251667,6.760171,0.476838,8.210126,14.395757,154.011419,188.472530,0.680250,0.572270,0.283484,0.225568,0.397179,0.500003
min,0.000000,10798.000000,30.000000,1.000000,55.000000,10.000000,-150.000000,-70.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,25006.750000,17664.000000,48.000000,1.000000,159.000000,65.000000,120.000000,80.000000,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000
50%,50001.500000,19703.000000,54.000000,1.000000,165.000000,72.000000,120.000000,80.000000,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000
75%,74889.250000,21327.000000,58.000000,2.000000,170.000000,82.000000,140.000000,90.000000,2.000000,1.000000,0.000000,0.000000,1.000000,1.000000
max,99999.000000,23713.000000,65.000000,2.000000,250.000000,200.000000,16020.000000,11000.000000,3.000000,3.000000,1.000000,1.000000,1.000000,1.000000


In [9]:
df.age_years.value_counts()

age_years
54    5961
56    5944
58    5693
60    5420
52    5279
50    5242
62    3887
64    3854
48    3167
46    2923
44    2805
40    2676
42    2602
55    1584
57    1478
53    1460
59    1378
51    1377
61    1225
49    1123
63    1078
47     793
45     790
41     777
43     733
39     470
65     277
30       4
Name: count, dtype: int64

In [4]:
df.gender.value_counts()

gender
1    45530
2    24470
Name: count, dtype: int64

In [12]:
df.height.value_counts()

height
165    5853
160    5022
170    4679
168    4399
164    3396
       ... 
112       1
98        1
74        1
119       1
57        1
Name: count, Length: 109, dtype: int64

In [13]:
df.weight.value_counts()

weight
65.0    3850
70.0    3764
68.0    2831
75.0    2740
60.0    2710
        ... 
92.2       1
75.6       1
82.9       1
23.0       1
99.9       1
Name: count, Length: 287, dtype: int64

In [14]:
df.ap_hi.value_counts()

ap_hi
 120     27699
 140      9506
 130      8961
 110      8644
 150      4450
         ...  
 1620        1
 199         1
-150         1
-115         1
 2000        1
Name: count, Length: 153, dtype: int64

In [15]:
df.ap_lo.value_counts()

ap_lo
80     34847
90     14316
70     10245
100     4082
60      2727
       ...  
56         1
182        1
810        1
9          1
602        1
Name: count, Length: 157, dtype: int64

In [16]:
df.cholesterol.value_counts()

cholesterol
1    52385
2     9549
3     8066
Name: count, dtype: int64

In [18]:
df.gluc.value_counts()

gluc
1    59479
3     5331
2     5190
Name: count, dtype: int64

In [5]:
df.smoke.value_counts()

smoke
0    63831
1     6169
Name: count, dtype: int64

In [6]:
df.alco.value_counts()

alco
0    66236
1     3764
Name: count, dtype: int64

In [7]:
df.active.value_counts()

active
1    56261
0    13739
Name: count, dtype: int64

In [8]:
df.cardio.value_counts()

cardio
0    35021
1    34979
Name: count, dtype: int64

# Prep Data for Training

In [ ]:
# Convert categorical data to numeric with `pd.get_dummies`
df_dummies = pd.get_dummies(df, columns=['cholesterol', 'gluc'], prefix=['cholesterol', 'gluc'])

In [6]:
# Split our preprocessed data into our features and target arrays
X = df.drop(columns=['cardio', 'age_years']).values
y = df['cardio'].values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [7]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [27]:
number_input_features = len(X_train[0])

# Create a method that creates a new Sequential model with hyperparameter options
def build_model(hp):
    # Create Model
    nn_model = tf.keras.models.Sequential()
    
    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation', ['relu','tanh','sigmoid', 'softmax', 'leaky_relu'])

    # Allow kerastuner to decide number of neurons in first layer
    nn_model.add(tf.keras.layers.Dense(
        units=hp.Int('first_units', min_value=1, max_value=4, step=2), 
        activation=activation, 
        input_dim=number_input_features))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 2)):
        nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=4,
            step=2),
            activation=activation))

    # Add dropout layer with a choice for dropout rate
    nn_model.add(tf.keras.layers.Dropout(
        hp.Float(f'dropout_{i}', min_value=0.0, max_value=0.5, step=0.2)))

    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nn_model.compile(loss="binary_crossentropy", optimizer=tf.keras.optimizers.Adam(learning_rate=hp.Choice('learning_rate', 
                                                                                                            values=[1e-2, 1e-3, 1e-4])), 
                                                                                                            metrics=["accuracy"])

    return nn_model

In [28]:
# Import the kerastuner library
import keras_tuner as kt

tuner = kt.Hyperband(
    build_model,
    objective="val_accuracy",
    max_epochs=20,
    hyperband_iterations=2)

In [29]:
%%time
# Run the kerastuner search for best hyperparameters
tuner.search(X_train_scaled,y_train,epochs=100,validation_data=(X_test_scaled,y_test))

Trial 60 Complete [00h 00m 35s]
val_accuracy: 0.7314285635948181

Best val_accuracy So Far: 0.7345714569091797
Total elapsed time: 00h 13m 25s
CPU times: total: 17min 45s
Wall time: 13min 25s


In [ ]:
# Get best model hyperparameters
best_hyper = tuner.get_best_hyperparameters(1)[0]
best_hyper.values

In [ ]:
# Evaluate best model against full test data
best_model = tuner.get_best_models(1)[0]
model_loss, model_accuracy = best_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

# LazyPredict

In [4]:
from lazypredict.Supervised import LazyClassifier
clf = LazyClassifier(verbose=1,ignore_warnings=False, custom_metric=None)
models,predictions = clf.fit(X_train_scaled, X_test_scaled, y_train, y_test)
models

  3%|▎         | 1/29 [00:01<00:36,  1.29s/it]

{'Model': 'AdaBoostClassifier', 'Accuracy': 0.7345428571428572, 'Balanced Accuracy': 0.7348239857552007, 'ROC AUC': 0.7348239857552006, 'F1 Score': 0.7332616761612806, 'Time taken': 1.2900631427764893}


  7%|▋         | 2/29 [00:03<00:57,  2.12s/it]

{'Model': 'BaggingClassifier', 'Accuracy': 0.6992, 'Balanced Accuracy': 0.6994162192309363, 'ROC AUC': 0.6994162192309363, 'F1 Score': 0.6983457929080357, 'Time taken': 2.700540542602539}
{'Model': 'BernoulliNB', 'Accuracy': 0.7149428571428571, 'Balanced Accuracy': 0.7151566996500516, 'ROC AUC': 0.7151566996500516, 'F1 Score': 0.7141532910975374, 'Time taken': 0.08806967735290527}


 14%|█▍        | 4/29 [00:04<00:21,  1.14it/s]

...

{'Model': 'SVC', 'Accuracy': 0.7263142857142857, 'Balanced Accuracy': 0.726439165883797, 'ROC AUC': 0.7264391658837971, 'F1 Score': 0.726063674435472, 'Time taken': 62.422999143600464}
StackingClassifier model failed to execute
StackingClassifier.__init__() missing 1 required positional argument: 'estimators'


100%|██████████| 29/29 [11:17<00:00, 16.91s/it]

{'Model': 'XGBClassifier', 'Accuracy': 0.7279714285714286, 'Balanced Accuracy': 0.7281244499911999, 'ROC AUC': 0.7281244499911999, 'F1 Score': 0.7275927027676745, 'Time taken': 0.27407050132751465}
[LightGBM] [Info] Number of positive: 17409, number of negative: 17591
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001687 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 934
[LightGBM] [Info] Number of data points in the train set: 35000, number of used features: 12
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.497400 -> initscore=-0.010400
[LightGBM] [Info] Start training from score -0.010400
{'Model': 'LGBMClassifier', 'Accuracy': 0.7379428571428571, 'Balanced Accuracy': 0.7381098669007276, 'ROC AUC': 0.7381098669007276, 'F1 Score': 0.7375070184401261, 'Time taken': 0.19223403930664062}


100%|██████████| 29/29 [11:17<00:00, 23.36s/it]


,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
LGBMClassifier,0.74,0.74,0.74,0.74,0.19
AdaBoostClassifier,0.73,0.73,0.73,0.73,1.29
XGBClassifier,0.73,0.73,0.73,0.73,0.27
SVC,0.73,0.73,0.73,0.73,62.42
RandomForestClassifier,0.73,0.73,0.73,0.73,8.23
SGDClassifier,0.72,0.72,0.72,0.72,0.10
ExtraTreesClassifier,0.72,0.72,0.72,0.72,4.53
LogisticRegression,0.72,0.72,0.72,0.72,0.09
BernoulliNB,0.71,0.72,0.72,0.71,0.09


In [8]:
clf_full = LazyClassifier(verbose=1,ignore_warnings=False, custom_metric=None)
models_full,predictions_full = clf_full.fit(X_train_scaled, X_test_scaled, y_train, y_test)
models_full

  3%|▎         | 1/29 [00:01<00:50,  1.79s/it]

{'Model': 'AdaBoostClassifier', 'Accuracy': 0.7358857142857143, 'Balanced Accuracy': 0.7363721223110291, 'ROC AUC': 0.7363721223110291, 'F1 Score': 0.7346996596255968, 'Time taken': 1.7923941612243652}


  7%|▋         | 2/29 [00:05<01:24,  3.13s/it]

{'Model': 'BaggingClassifier', 'Accuracy': 0.7009714285714286, 'Balanced Accuracy': 0.701328534294019, 'ROC AUC': 0.701328534294019, 'F1 Score': 0.7002565760846645, 'Time taken': 4.059341669082642}
{'Model': 'BernoulliNB', 'Accuracy': 0.7166857142857143, 'Balanced Accuracy': 0.7170189090048812, 'ROC AUC': 0.7170189090048812, 'F1 Score': 0.7161020805207002, 'Time taken': 0.06246590614318848}


 14%|█▍        | 4/29 [00:06<00:31,  1.27s/it]

...

 97%|█████████▋| 28/29 [1:05:08<00:52, 52.29s/it]

{'Model': 'XGBClassifier', 'Accuracy': 0.7342285714285715, 'Balanced Accuracy': 0.7344683010109269, 'ROC AUC': 0.734468301010927, 'F1 Score': 0.7339566108050288, 'Time taken': 0.2284839153289795}
[LightGBM] [Info] Number of positive: 26167, number of negative: 26333
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001644 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 979
[LightGBM] [Info] Number of data points in the train set: 52500, number of used features: 12
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.498419 -> initscore=-0.006324
[LightGBM] [Info] Start training from score -0.006324


100%|██████████| 29/29 [1:05:08<00:00, 134.77s/it]

{'Model': 'LGBMClassifier', 'Accuracy': 0.7388571428571429, 'Balanced Accuracy': 0.7391274737111373, 'ROC AUC': 0.7391274737111373, 'F1 Score': 0.738512956758573, 'Time taken': 0.23040199279785156}


,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
LGBMClassifier,0.74,0.74,0.74,0.74,0.23
AdaBoostClassifier,0.74,0.74,0.74,0.73,1.79
XGBClassifier,0.73,0.73,0.73,0.73,0.23
SVC,0.73,0.73,0.73,0.73,80.15
SGDClassifier,0.73,0.73,0.73,0.73,0.14
RandomForestClassifier,0.72,0.73,0.73,0.72,12.12
LogisticRegression,0.72,0.72,0.72,0.72,0.16
BernoulliNB,0.72,0.72,0.72,0.72,0.06
ExtraTreesClassifier,0.72,0.72,0.72,0.72,5.93
